In [ ]:
!git clone https://github.com/Ayan-Ansar/Eng2HingTranslator.git

In [2]:
%cd '/content/Eng2HingTranslator'

/content/Eng2HingTranslator


In [3]:
# If model is not cloned from the git repository you can download the trained model zip file by executing this cell
#!gdown '1eTjR5Nw4vJmH4crKuTUM2XRpAwOqg-gd' --output '/content/Eng2HingTranslator/drive_saved_model.zip'
#!unzip '/content/Eng2HingTranslator/drive_saved_model.zip' -d '/content/Eng2HingTranslator/drive_saved_model'

Downloading...
From: https://drive.google.com/uc?id=1eTjR5Nw4vJmH4crKuTUM2XRpAwOqg-gd
To: /content/Eng2HingTranslator/drive_saved_model.zip
100% 346M/346M [00:05<00:00, 69.2MB/s]
Archive:  /content/Eng2HingTranslator/drive_saved_model.zip
  inflating: /content/Eng2HingTranslator/drive_saved_model/tf_model.h5  
  inflating: /content/Eng2HingTranslator/drive_saved_model/generation_config.json  
  inflating: /content/Eng2HingTranslator/drive_saved_model/config.json  


In [ ]:
!pip install -r requirements.txt

### import dependencies

In [5]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
import numpy as np
import nltk
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
nltk.download('words')
from nltk.corpus import words
import warnings

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [6]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Load Pretrained model

In [7]:
model_checkpoint = "t5-small"

# if you are using trained model download from gdrive, uncomment the line below and comment the line after that
#model_path = '/content/Eng2HingTranslator/drive_saved_model'
model_path = '/content/Eng2HingTranslator/saved_models/'

In [8]:
# load the tf-small tokenizer and the pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_path)

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

(…)ce.co/t5-small/resolve/main/spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…).co/t5-small/resolve/main/tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/Eng2HingTranslator/drive_saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


### Inference

In [9]:
# you can add more common words that can be in both english and hindi dictonary
common_hindi_eng_words = ['to','agar','me','hoon','ye','ka','mein','ko']

In [11]:
def translate_to_hindi(word):
    try:
        translation = transliterate(word, sanscript.ITRANS, sanscript.DEVANAGARI)
        return translation
    except Exception as e:
        return word

def translate(input_text):
    tokenized = tokenizer([input_text], return_tensors='np')
    out = model.generate(**tokenized, max_length=128)
    with tokenizer.as_target_tokenizer():
        result = tokenizer.decode(out[0], skip_special_tokens=True)
    print('Translated:', result)
    result = result.lower().split(' ')
    ans = []
    for i in result:
        if i in common_hindi_eng_words or i not in words.words():
            i = translate_to_hindi(i)
        ans.append(i)

    return ' '.join(ans)

In [12]:
INPUT = ["Definitely share your feedback in the comment section",
         "So even if it's a big video, I will clearly mention all the products",
         "I was waiting for my bag"]

In [13]:
for i, text in enumerate(INPUT):
    print(f"Input Text {i+1}: ", text)
    output_text = translate(text)
    print(f"Converted to mix enlgish-hindi Output Text {i+1}: ", output_text)
    print('\n')

Input Text 1:  Definitely share your feedback in the comment section


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Translated: Definitely apke comment section me share kare
Converted to mix enlgish-hindi Output Text 1:  definitely अप्के comment section मे share करे


Input Text 2:  So even if it's a big video, I will clearly mention all the products
Translated: To agar ye bada video hai to main sabhi products ko mention karunga
Converted to mix enlgish-hindi Output Text 2:  तो अगर् ये बद video है तो main सभि प्रोदुच्त्स् को mention करुन्ग


Input Text 3:  I was waiting for my bag
Translated: mai apna bag ka wait kar raha hoon
Converted to mix enlgish-hindi Output Text 3:  मै अप्न bag क wait कर् रह हून्


